In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Load data
df = pd.read_csv(r".\Data\train_data.csv")
ts_df = pd.read_csv(r".\Data\test_data.csv")

In [ ]:
# Feature processing
def preprocess_data(df):
    features = ["temp", "atemp", "hum", "windspeed", "season", "weekday", "hr", "holiday", "weathersit", "workingday"]
    target = "cnt"
    df_features = df[features]
    df_features.dropna(inplace=True)
    df_target = df[target]
    return df_features.values, df_target.values

X, y = preprocess_data(df)
scaler = MinMaxScaler(feature_range=(0, 1))
y = scaler.fit_transform(y.reshape(-1, 1))
tX, ty = preprocess_data(ts_df)
ty = scaler.fit_transform(ty.reshape(-1, 1))
print(len(df))

In [ ]:
# Dataset definition
class BikeDataset(Dataset):
    def __init__(self, X, y, input_len=96, output_len=96):
        self.X = X
        self.y = y
        self.input_len = input_len
        self.output_len = output_len

    def __len__(self):
        return len(self.X) - self.input_len - self.output_len + 1

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx:idx + self.input_len], dtype=torch.float32),
            torch.tensor(self.y[idx + self.input_len:idx + self.input_len + self.output_len], dtype=torch.float32),
        )

In [ ]:
# Create data loaders
train_dataset = BikeDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = BikeDataset(tX, ty)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
# Transformer model
class TransformerModel(nn.Module):
        def __init__(self, input_dim, output_dim, seq_len, hidden_dim=512, nhead=4, num_encoder_layers=3):
            super(TransformerModel, self).__init__()
            self.embedding = nn.Linear(input_dim, hidden_dim)
            self.pos_encoder = nn.Parameter(torch.randn(seq_len, hidden_dim)).unsqueeze(0).repeat(32, 1, 1).permute(1,0,2)
            self.transformer = nn.Transformer(d_model=hidden_dim, nhead=nhead, num_encoder_layers=num_encoder_layers, dropout=0.3, activation="gelu")
            self.fc_out = nn.Linear(hidden_dim, output_dim)

        def forward(self, x):
            x = x.permute(1, 0, 2)
            device = x.device
            self.pos_encoder = self.pos_encoder.to(device)
            x = self.embedding(x)+self.pos_encoder
            x = self.transformer(x, x)
            return self.fc_out(x[-1, :, :])
def get_transformer_model(input_dim, output_dim, seq_len):
    return TransformerModel(input_dim, output_dim, seq_len)

model = get_transformer_model(input_dim=X.shape[1], output_dim=96, seq_len=96)
print(X.shape[1])
model = model.to(device)

In [ ]:
# Training setup
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
def train_model(model, dataloader, criterion, optimizer, epochs=30):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for inputs, targets in dataloader:
            # print(type(targets))
            # inputs, targets = pad_sequence(inputs), pad_sequence(targets)
            if len(inputs) != 32:
                continue
            targets = targets.to(device)
            inputs = inputs.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            # print(outputs.shape, targets[:,:,0].shape)
            loss = criterion(outputs.squeeze(), targets[:,:,0].squeeze())
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / (len(dataloader)//32*32):.4f}")

train_model(model, train_loader, criterion, optimizer)
# 保存整个模型
torch.save(model, "Transformer96-4.pth")


In [ ]:
# Plotting predictions vs actual values
model.eval()
predictions, actuals = [], []
cnt = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        # inputs = inputs.transpose(0, 1)
        inputs = inputs.to(device)
        if len(inputs)!=32:
            continue
        outputs = model(inputs)
        predictions.append(outputs.cpu().numpy())
        actuals.append(targets.cpu().numpy())

In [ ]:
plt.plot(actuals[0][0], label="Actual")  # 第一个批次，第一个样本的真实值
plt.plot(predictions[0][0], label="Prediction")  # 第一个批次，第一个样本的预测值
plt.legend()
plt.title("Prediction vs Actual")
plt.show()